In [9]:
from elasticsearch import Elasticsearch

# 数据来源:data/example_book_data.jsonl
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

A query that matches documents matching boolean combinations of other queries. The bool query maps to Lucene BooleanQuery. It is built using one or more boolean clauses, each clause with a typed occurrence. The occurrence types are:

| Occur      | Description                                                  |
| ---------- | ------------------------------------------------------------ |
| `must`     | The clause (query) must appear in matching documents and will contribute to the score. |
| `filter`   | The clause (query) must appear in matching documents. However unlike `must` the score of the query will be ignored. Filter clauses are executed in [filter context](https://www.elastic.co/guide/en/elasticsearch/reference/7.11/query-filter-context.html), meaning that scoring is ignored and clauses are considered for caching. |
| `should`   | The clause (query) should appear in the matching document.   |
| `must_not` | The clause (query) must not appear in the matching documents. Clauses are executed in [filter context](https://www.elastic.co/guide/en/elasticsearch/reference/7.11/query-filter-context.html) meaning that scoring is ignored and clauses are considered for caching. Because scoring is ignored, a score of `0` for all documents is returned. |

In [10]:
body = {
    "query": {
        "bool": {
            "must": [
                {"term":
                    {"name": {
                        "value": "十一五"
                    }
                    }
                },
                {"term":
                    {"type":
                        {
                            "value": "大学"
                        }
                    }
                }
            ],
            "filter": [
                # 也可以使用脚本进行过滤
                {
                    "match": {
                        "name": {
                            "query": "国家级"
                        }
                    }
                }
            ],
            "must_not": [
                {
                    "range": {
                        "price": {
                            "gte": 0,
                            "lte": 25
                        }
                    }
                }],
            "should": [
                {"term":
                    {"name":
                        {
                            "value": "英语"
                        }
                    }
                },
                {"term":
                    {"name":
                        {
                            "value": "医学"
                        }
                    }
                },
                {"match":
                    {"info":
                        {
                            "query": "数学"
                        }
                    }
                }
            ],
            "minimum_should_match": 1
        }
    }
}

'''
Using minimum_should_match
You can use the minimum_should_match parameter to specify the number or percentage of should clauses returned documents must match.

If the bool query includes at least one should clause and no must or filter clauses, the default value is 1. Otherwise, the default value is 0.
'''
'''
GET test/_search
`body`
'''
es.search(index='test', body=body)

{'took': 5,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 16, 'relation': 'eq'},
  'max_score': 7.3981476,
  'hits': [{'_index': 'test',
    '_type': '_doc',
    '_id': '322',
    '_score': 7.3981476,
    '_source': {'name': '普通高等教育十一五国家级规划教材・医学化学',
     'publish': '高等教育出版社',
     'type': '大学教材',
     'author': ['徐春祥'],
     'info': '《普通高等教育十一五国家级规划教材・医学化学》是普通高等教育“十一五”国家级规划教材。《普通高等教育十一五国家级规划教材・医学化学》包括无机化学、有机化学、定量分析化学的基本内容及实验。全书包括22章和20个实验，理论内容在前，实验内容在后。《普通高等教育十一五国家级规划教材・医学化学》既重视医学化学中的重要概念和基本知识，又吸收了国内外近几年出版的无机化学、有机化学和分析化学教材的新内容，同时也选择了一些与医学关系密切的相关内容。',
     'price': 31}},
   {'_index': 'test',
    '_type': '_doc',
    '_id': '52',
    '_score': 7.25508,
    '_source': {'name': '普通高等教育“十一五”国家级规划教材・医用高等数学（第2版）',
     'publish': '高等教育出版社',
     'type': '大学教材',
     'author': ['乐经良', '祝国强'],
     'info': '《普通高等教育“十一五”国家级规划教材・医用高等数学（第2版）》包括函数与极限，导数与微分，不定积分，定积分及其应用，微分方程，多元函数微积分，概率论初步及线性代数基础等共八章。本版是在教材第一版的基础上，按照教育部非数学类专业数